Выполнить поиск по артисту

In [21]:
import requests, json, urllib.parse, dateutil.parser, statistics

search_artist = input("введите артиста: ")

requests_params = {
    'term':  search_artist,
    'limit': 50,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

request_data = json.loads( requests.get( f"https://itunes.apple.com/search?{urllib.parse.urlencode(requests_params)}" ).text )

#print( request_data['results'] )

ConnectTimeout: HTTPSConnectionPool(host='itunes.apple.com', port=443): Max retries exceeded with url: /search?term=oxymiron&limit=50&media=music&entity=song&attribute=artistTerm (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000014C1F658430>, 'Connection to itunes.apple.com timed out. (connect timeout=None)'))

Выполнить первичную очитску данных

In [22]:
сlear_request_data = list( map( lambda anon: {**anon, 'releaseDate': dateutil.parser.isoparse(anon['releaseDate'])}, request_data['results'] ) )

сlear_request_data = list( map( lambda anon: {
    'artistName': anon['artistName'],
    'kind' : anon['kind'],
    'artistId': anon['artistId'],
    'collectionId': anon['collectionId'],
    'trackId': anon['trackId'],
    'artistName': anon['artistName'],
    'collectionName': anon['collectionName'],
    'trackName': anon['trackName'],
    'collectionPrice': anon['collectionPrice'],
    'trackPrice': anon['trackPrice'],
    'releaseDate': anon['releaseDate'],
    'trackCount': anon['trackCount'],
    'trackNumber': anon['trackNumber'],
    'trackTimeMillis': anon['trackTimeMillis'],
    'country': anon['country'],
    'currency': anon['currency'],
    'primaryGenreName': anon['primaryGenreName']
}, сlear_request_data ) )

#print(сlear_request_data)

Немного функций для удобства

In [23]:
def convert_millis(millis):
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = ( total_seconds // 60 ) % 60
    hours = ( total_seconds // ( 60 * 60 ) )
    return f"{hours}:{minutes}:{seconds}"

    # С удовольствием сделал бы так но в задании четко написано {часы}:{минуты}:{секунды}
    #if hours > 0:
    #    return f"{hours} часов {minutes} минут {seconds} секунд"
    #if minutes > 0: 
    #    return f"{minutes} минут {seconds} секунд"
    #return f"{seconds} секунд"

def frequency_filter(filter_list):
    filter_result = { i : filter_list.count(i) for i in filter_list }
    return max( filter_result, key = filter_result.get )

Собрать статистику по исполнителю из полученной выборки данных

In [27]:
print( 'название исполнителя:', frequency_filter( list( map( lambda anon: anon['artistName'], сlear_request_data ) ) ) )

print( 'количество песен:', len( list( map( lambda anon: anon['trackName'], сlear_request_data ) ) ) )

print( 'популярный жанр:', frequency_filter( list( map( lambda anon: anon['primaryGenreName'], сlear_request_data ) ) ) )

print( 'количество альбомов:', len( list( set( map( lambda anon: anon['collectionName'], сlear_request_data ) ) ) ) )

print( 'общая длительность песен:', convert_millis( sum( list( map( lambda anon: anon['trackTimeMillis'], сlear_request_data ) ) ) ) )

print( 'средняя длительность песен:', convert_millis( round( statistics.mean( list( map( lambda anon : anon['trackTimeMillis'], сlear_request_data ) ) ) ) ) )

print( 'медиана длительности песен:', convert_millis( round( statistics.median( list( map( lambda anon : anon['trackTimeMillis'], сlear_request_data ) ) ) ) ) )

print( 'название самой долгой песни:', max( сlear_request_data, key = lambda anon: anon['trackTimeMillis'] )['trackName'] )

print( 'длительность самой долгой песни:', convert_millis( max( сlear_request_data, key = lambda anon: anon['trackTimeMillis'] )['trackTimeMillis'] ) )

print( 'название самой короткой песни:', min( сlear_request_data, key = lambda anon: anon['trackTimeMillis'] )['trackName'] )

print( 'длительность самой короткой песни:', convert_millis( min( сlear_request_data, key = lambda anon: anon['trackTimeMillis'] )['trackTimeMillis'] ) )

print( 'общая стоимость песен: ≈', round( sum( list( map( lambda anon: anon['trackPrice'], сlear_request_data ) ) ), 2 ), '$' )

print( 'средняя стоимость песен: ≈', round( statistics.mean( list( map( lambda anon: anon['trackPrice'], сlear_request_data ) ) ), 2 ), '$' )

print( 'медиана стоимости песен: ≈', round( statistics.median( list( map( lambda anon: anon['trackPrice'], сlear_request_data ) ) ), 2 ), '$' )

print( 'название самой дорогой песни:', max( сlear_request_data, key = lambda anon: anon['trackPrice'] )['trackName'] )

print( 'стоимость самой дорогой песни:', max( сlear_request_data, key = lambda anon: anon['trackPrice'] )['trackPrice'], '$' )

print( 'название самой дешевой песни:', min( сlear_request_data, key = lambda anon: anon['trackPrice'] )['trackName'] )

print( 'стоимость самой дешевой песни:', min( сlear_request_data, key = lambda anon: anon['trackPrice'] )['trackPrice'], '$' )

название исполнителя: Oxxxymiron
количество песен: 50
популярный жанр: Hip-Hop
количество альбомов: 11
общая длительность песен: 2:35:33
средняя длительность песен: 0:3:6
медиана длительности песен: 0:2:52
название самой долгой песни: KONSTRUKT
длительность самой долгой песни: 0:8:1
название самой короткой песни: Интро
длительность самой короткой песни: 0:1:22
общая стоимость песен: ≈ 49.8 $
средняя стоимость песен: ≈ 1.0 $
медиана стоимости песен: ≈ 0.99 $
название самой дорогой песни: ОЙДА
стоимость самой дорогой песни: 1.29 $
название самой дешевой песни: Полигон
стоимость самой дешевой песни: 0.99 $


Собрать статистику по альбомам исполнителя из полученной выборки данных

In [44]:
print(сlear_request_data['collectionName']);
x = сlear_request_data
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
print(get_indexes("Полигон", x))


TypeError: list indices must be integers or slices, not str